In [6]:
import json

In [7]:
 with open(f"/kaggle/input/wikipersonallru/person_all.json", encoding='utf-8') as json_file:
        data = json.load(json_file)
        for item_name, item_value in data.items():
            if 'Примечания' in item_value:
                del item_value['Примечания']
            if 'Ссылки' in item_value:
                del item_value['Ссылки']

In [8]:
data_1 = {}
for item_name, item_value in data.items():
    temp = []
    for headline, val in item_value.items():
        if val != []:
            temp.append(val[0])
    data_1[item_name] = temp

In [9]:
data_2 = []
for item_name, item_value in data_1.items():
    temp = ' '.join(item_value)
    data_2.append(temp)

### Libs installation

In [10]:
pip install pymystem3

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install bigartm

     |████████████████████████████████| 1.9 MB 4.6 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install nlp

     |████████████████████████████████| 1.7 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 24.1 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Imports

In [13]:
import nltk
import nlp
from nltk.corpus import stopwords
from pymystem3 import Mystem
import string
import re
from biased_lexrank_w2v import BiasedLexRank
import artm
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize

### Obtain dataset

In [14]:
nltk.download("stopwords")
sw = stopwords.words("english")
porter = PorterStemmer()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
def preprocess_text_eng(text):
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in sw and t != " "]
    #tokens = [t for t in tokens if t not in string.digits]
    tokens = [t for t in tokens if t == '.' or t not in string.punctuation]
    tokens = [porter.stem(t) for t in tokens]
    text = " ".join(tokens)
    return text

In [16]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def preprocess_text_rus(text):
    text = re.sub("[^а-яА-Я\s]+", ' ', text)
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " and not '\n' in token]
    text = " ".join(tokens)
    text = re.sub(" +", " ", text)
    return text


<>:5: DeprecationWarning: invalid escape sequence \s
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [17]:
data = [preprocess_text_rus(t) for t in data_2]

In [18]:
for item in data:
    item = item.replace('\xa0', '')

In [19]:
sw = russian_stopwords

### Topic modelling

In [20]:
cv = CountVectorizer(max_features=2000, stop_words=sw)
n_wd = np.array(cv.fit_transform(data).todense()).T
vocabulary = cv.get_feature_names()

In [21]:
bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=vocabulary)

In [22]:
if 1:
    model = artm.ARTM(num_topics=50, dictionary=bv.dictionary, cache_theta=True)
    
    model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=bv.dictionary))
    model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=15))

    model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-2))
    model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='decorrelator_phi_regularizer', tau=1e+5))

if 0:
    model = artm.LDA(num_topics=3, alpha=0.01, beta=0.001, cache_theta=True, num_document_passes=5, dictionary=bv.dictionary)

In [23]:
%%time
model.fit_offline(bv, num_collection_passes=35)

CPU times: user 2min 5s, sys: 570 ms, total: 2min 6s
Wall time: 37.8 s


In [24]:
#if artm
topics = {}
for topic_name in model.topic_names:
    print(topic_name + ': ')
    for word in model.score_tracker["top_tokens_score"].last_tokens[topic_name]:
        print(word, end=',')
    topics[topic_name] = model.score_tracker["top_tokens_score"].last_tokens[topic_name]
    print()

topic_0: 
работа,время,новый,получать,также,результат,развитие,создание,период,опыт,материал,качество,общий,основной,проводить,
topic_1: 
август,январь,июль,июнь,сентябрь,ноябрь,май,февраль,апрель,март,октябрь,декабрь,день,состояться,подписывать,
topic_2: 
сын,отец,время,брат,дочь,мать,жена,ребенок,старший,семья,умирать,род,брак,младший,сестра,
topic_3: 
страна,принимать,организация,участие,руководство,деятельность,борьба,активный,лидер,национальный,решение,греческий,греция,против,поддерживать,
topic_4: 
немецкий,германия,сниматься,еврейский,фон,берлин,немец,еврей,драма,германский,гитлер,немой,карл,мировой,рядом,
topic_5: 
наука,история,культура,государственный,член,международный,гг,медицинский,являться,доктор,статья,тема,общество,академия,исторический,
topic_6: 
дело,суд,заявлять,арестовывать,право,убийство,тюрьма,обвинение,власть,свобода,заключение,арест,полиция,обвинять,судебный,
topic_7: 
король,генрих,двор,герцог,престол,аль,вильгельм,королевство,владение,смерть,карл,короть,короле

In [ ]:
#if lda
top_tokens = model.get_top_tokens(num_tokens=10)
for i, token_list in enumerate(top_tokens):
     print('Topic #{0}: {1}'.format(i, token_list))

In [ ]:
theta = model.transform(batch_vectorizer=bv)
theta